In [6]:
import json
import os
import pandas as pd

from dotenv import load_dotenv
from openrouteservice import client, places, directions

In [7]:
output_dir = '../data/curated/'

## set variables
load_dotenv()
tokens = []
for i in range(1,17): # number of keys
    token = os.environ.get(f'token_{i}')
    tokens.append(token)
category_ids = [604] # railway station
buffer_size = 2000   # between 1-2000m
ptv_dict = {}

In [8]:
## read apartment data
property_data = json.load(open( f"{output_dir}properties_processed.json" ))

In [229]:
token = tokens[0]

## set x and y
x = len(ptv_dict)
y = len(ptv_dict)+500

for index in range(x,y):
    if index > len(property_data['Coordinates'].keys()):
        break
    backwards = list(map(float,property_data['Coordinates'][str(index)][1:-1].split(',')))
    coords = [backwards[1], backwards[0]]

    ## query code
    ors = client.Client(key=token)
    query = {'request': 'pois',
            'geojson': {'type':'Point','coordinates':coords},
            'buffer': buffer_size,
            'filter_category_ids': category_ids,
            'sortby':'distance',
            'limit': 2}
    
    features = ors.places(**query)['features']
    destination = features[0]['geometry']['coordinates']
    all = [(str(coords[0]),str(coords[1])),(str(destination[0]),str(destination[1]))]

    request = {'coordinates': all,
            'profile': 'driving-car',
            'geometry': 'true',
            'format_out':'geojson'}
    route = ors.directions(**request)
    info = route['features'][0]['properties']['summary']

    ## add information
    ptv_dict[str(index)] = info

KeyError: '7526'

In [230]:
print(len(ptv_dict))

7526


In [231]:
property_data['PTV'] = ptv_dict

## write json
json.dump(property_data, open(f"{output_dir}num_ptv.json", 'w'))

# write csv
data = pd.read_json(f"{output_dir}num_ptv.json")
data.to_csv(f"{output_dir}num_ptv.csv")